## Import

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
plt.rcParams['font.family'] ='Malgun Gothic'
plt.rcParams['axes.unicode_minus'] =False

In [ ]:
data = pd.read_csv("data.csv",encoding='cp949')
quality = pd.read_excel("quality.xlsx")
train = pd.read_csv("train.csv",encoding='cp949')

## Data Info

#### 일관성

In [ ]:
data['TAG_MIN'] = pd.to_datetime(data['TAG_MIN'],format='%Y-%m-%d',errors='coerce')
data['TAG_MIN'].isnull().sum()

In [ ]:
data.describe()

In [ ]:
data.info()

In [ ]:
quality.describe()

In [ ]:
quality.info()

In [ ]:
train.describe()

In [ ]:
train.info()

#### 완전성

In [ ]:
gds = data.groupby('배정번호')
f_num = [group_df for _, group_df in gds]

In [ ]:
for i in range(len(data.columns[2:])):
    exec(f'list_{i}=[]')

for i in range(len(gds)):
    for j in range(len(data.columns[2:])):
        exec(f'list_{j}.append((1-(f_num[{i}].isna().sum()["{data.columns[2:][j]}"]/len(f_num[{i}])))*100)') 

In [ ]:
plt.figure(figsize=(9,6))
for i in range(len(data.columns[2:])):
    plt.plot(eval(f'list_{i}'),label=f'{data.columns[2:][i]} 완전성')
plt.plot([30 for i in range(len(list_0))],label='기준(30%)',c='r')
plt.xlabel('배정번호 index')
plt.ylabel('완전성(%)')
plt.legend()
plt.show()

In [ ]:
non_first_count = 0
for i in range(len(f_num)):    
    na_row = f_num[i].isna().sum(axis=1)
    non_first_count += sum(f_num[i].index[:sum(na_row!=0)] != na_row[na_row!=0].index)

non_first_count

In [ ]:
plt.figure(figsize=(9,6))
for i in range(3):
    na_row = f_num[i].isna().sum(axis=1)
    plt.plot(na_row, label = f"배정번호 {quality['배정번호'].iloc[i]} 결측치 개수")

plt.xlabel('시간 index')
plt.ylabel('결측 특성 개수')
plt.legend()
plt.show()


In [ ]:
isna_list = []
for i in range(len(f_num)):
    isna_list.append((1-(f_num[i].isna().sum().max()/len(f_num[i])))*100)

In [ ]:
plt.figure(figsize=(9,6))
plt.plot(isna_list,label= '배정번호별 완전성')
plt.plot([30 for i in range(len(isna_list))],label='기준(30%)',c='r')
plt.xlabel('배정번호 index')
plt.ylabel('완전성(%)')
plt.legend()
plt.show()

In [ ]:
np.array(isna_list)[np.array(isna_list)>0.3]

#### 유일성

In [ ]:
sum(quality.groupby(['배정번호']).size()==1)

In [ ]:
sum(quality.groupby(['배정번호']).size()==1)/len(quality)*100

#### 정확성

In [ ]:
quality['추정총수량'] = quality['양품수량'] + quality['불량수량']
(1-sum(quality['추정총수량'] != quality['총수량'])/len(quality))*100

In [ ]:
quality['데이터수'] = 0
for i in quality['배정번호']:
    quality.loc[quality['배정번호']==i, '데이터수'] = data.groupby('배정번호')['TAG_MIN'].count().loc[i]

In [ ]:
quality['오차율'] = abs(quality['총수량'] - quality['데이터수'])/quality['총수량']*100

In [ ]:
plt.figure(figsize=(9,6))
plt.plot(quality['오차율'],label='배정번호별 총수량 대비 데이터수 오차율')
plt.xlabel('배정번호 index')
plt.ylabel('오차율(%)')
plt.legend()
plt.show()

In [ ]:
quality['불량률'] = quality['불량수량']/quality['총수량']

#### 유효성

In [ ]:
sum(quality['배정번호'].unique() == data['배정번호'].unique())

In [ ]:
sum(quality['배정번호'].unique() == data['배정번호'].unique())/max(len(quality['배정번호'].unique()),len(data['배정번호'].unique()))*100

In [ ]:
list_colname = []
for i in train.columns[1:]:
    list_colname.append(i.split('_')[0])

set(data.columns[2:]) - set(list_colname)

In [ ]:
col = np.array([[i + '_Avg', i + '_Std'] for i in data.drop(['소입로 CP 모니터 값'],axis=1).columns[2:]]).reshape(-1).tolist()
meanstd_df = pd.DataFrame(np.array(data.drop(['소입로 CP 모니터 값'],axis=1).iloc[:,1:].groupby('배정번호').agg(['mean','std'])),columns=col)
meanstd_df = pd.concat([pd.DataFrame(data['배정번호'].unique(),columns=['배정번호']),meanstd_df],axis=1)
meanstd_df.iloc[list(train['Unnamed: 0']),:]

In [ ]:
train

In [ ]:
(np.abs(np.array(meanstd_df.iloc[list(train['Unnamed: 0']),1:]) - np.array(train.iloc[:,1:-1]))/np.array(train.iloc[:,1:-1])).mean()*100

In [ ]:
train['불량률'] = 0
for i in range(len(train)):
    n = train['Unnamed: 0'].iloc[i]
    train.loc[train['Unnamed: 0'] == n, '불량률'] = quality['불량률'].iloc[n]

train.groupby('불량단계')['불량률'].describe()

In [ ]:
quality['불량단계'] = 0
quality.loc[quality['불량률']>=0.0004525, '불량단계'] = 1


## EDA

In [ ]:
quality = quality.drop(['작업일','공정명','설비명','양품수량','불량수량','총수량','추정총수량','오차율'],axis=1)
quality.sort_values('불량률')

In [ ]:
low_list = [quality.sort_values('불량률')['배정번호'].iloc[i] for i in range(6)]
high_list = [quality.sort_values('불량률',ascending=False)['배정번호'].iloc[i] for i in range(6)]

low_list, high_list

In [ ]:
for i in data.columns[2:]:
    plt.figure(figsize=(16,9))
    for j in range(6):
        plt.plot(data.loc[data['배정번호'] == low_list[j], i].reset_index(drop=True), label = f'배정번호 {low_list[j]}(불량률 낮)',c=f'#a{j+4}0000')
    for k in range(6):
        plt.plot(data.loc[data['배정번호'] == high_list[k], i].reset_index(drop=True), label = f'배정번호 {high_list[k]}(불량률 높)',c=f'#0000a{k+4}')
    plt.xlabel('시간 index')
    plt.ylabel(i)
    plt.legend()
    plt.show()

In [ ]:
index = 0
for i in data.columns[2:]:
    for ii in data.columns[3+index:]:
        plt.figure(figsize=(16,9))
        for j in range(6):
            plt.scatter(x = data.loc[data['배정번호'] == low_list[j], i].reset_index(drop=True), y = data.loc[data['배정번호'] == low_list[j], ii].reset_index(drop=True), label = f'배정번호 {low_list[j]}(불량률 낮)',c=f'#a{j+4}0000',alpha=0.5)
        for k in range(6):
            plt.scatter(x = data.loc[data['배정번호'] == high_list[k], i].reset_index(drop=True), y = data.loc[data['배정번호'] == high_list[k], ii].reset_index(drop=True), label = f'배정번호 {high_list[k]}(불량률 높)',c=f'#0000a{k+4}',alpha=0.5)
        plt.xlabel(i)
        plt.ylabel(ii)
        plt.legend()
        plt.show()
    index += 1

In [ ]:
plt.figure(figsize=(16,9))
plt.plot(quality['불량률'],label='배정번호별 불량률')
plt.plot([0.0004525 for i in range(len(quality['불량률']))], label = '불량 단계 위험 기준(0.0004525)',c='r')
plt.xlabel('배정번호 index')
plt.ylabel('불량률')
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(16,9))
corr_data_high = data[data['배정번호']==high_list[0]].drop(['TAG_MIN','배정번호'],axis=1).corr(method='pearson').round(2)
sns.heatmap(corr_data_high, annot=True, annot_kws={"size":10})

In [ ]:
plt.figure(figsize=(16,9))
corr_data_low = data[data['배정번호']==low_list[0]].drop(['TAG_MIN','배정번호'],axis=1).corr(method='pearson').round(2)
sns.heatmap(corr_data_low, annot=True, annot_kws={"size":10})

In [ ]:
plt.figure(figsize=(16,9))
sns.heatmap((corr_data_high - corr_data_low), annot=True, annot_kws={"size":10})

## Data Preprocessing

In [ ]:
data = pd.read_csv("data.csv",encoding='cp949')
quality = pd.read_excel("quality.xlsx")

In [ ]:
data = data.dropna()
data = data.drop(['TAG_MIN'],axis=1)
data

In [ ]:
quality['불량률'] = quality['불량수량']/quality['총수량']
quality['불량단계'] = 0
quality.loc[quality['불량률']>=0.0004525, '불량단계'] = 1
quality['데이터수'] = data.groupby('배정번호')['세정기'].count().reset_index(drop=True)
quality['위험군개수'] = np.round(quality['불량률']*quality['데이터수']*10)
quality = quality.drop(['작업일','공정명','설비명','양품수량','불량수량','총수량','불량률'],axis=1)
quality

In [ ]:
data.to_csv("preprocess_data.csv",index=False)
quality.to_csv("preprocess_quality.csv",index=False)